In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data_path = 'rct.csv'

df = pd.read_csv(data_path)

In [3]:
# drops weird moisture values
df['cull'] = df['cull'].astype(float)
df = df[df.moisture.str.contains('#DIV/0!') == False]
df = df[df.tsi.str.contains('#DIV/0!') == False]
# drops empty and 0 value rows
df = df[np.isfinite(df['stfi'])]
df = df.loc[(df != 0).all(axis=1), :]
#only get usp 120
#df = df[df.grade.str.contains('NL050550')]
df = df.loc[df['cull'] == 69.5]
df = df.reset_index(drop=True)

In [4]:
df[0:200]

,grade,label,rct,tsi,stfi,caliper,moisture,basisweight,cull
0,NL050350,1/3/2017 12:40,76.0,4.56,19.81,10.60,6.74,34.89,69.5
1,NL050350,1/3/2017 10:48,74.0,4.60,19.30,10.64,7.14,35.26,69.5
2,NL050350,1/3/2017 14:32,77.0,4.63,20.11,10.53,7.04,35.28,69.5
3,NL050350,1/3/2017 16:25,76.0,4.71,20.25,10.61,6.80,35.34,69.5
4,NL050350,1/3/2017 8:59,83.0,4.74,17.52,11.25,6.35,36.37,69.5
5,NL050350,3/21/2017 5:22,73.0,4.52,20.51,10.11,7.82,36.81,69.5
6,NL050350,3/21/2017 2:40,76.0,4.75,19.69,10.06,6.79,36.90,69.5
7,NL050350,3/21/2017 7:11,71.0,4.57,20.76,9.92,7.65,36.92,69.5
8,NL050350,3/20/2017 23:57,72.0,4.61,21.42,10.37,7.43,37.58,69.5
9,NL050350,3/20/2017 21:14,78.0,4.60,21.30,10.54,7.07,38.03,69.5


In [5]:
df['tsi'] = df['tsi'].astype(float)
df['moisture'] = df['moisture'].astype(float)
df['cull'] = df['cull'].astype(float)

In [6]:
# make binary numbers for grades
'''
dummy_fields = ['grade']
for each in dummy_fields:
    dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
    df = pd.concat([df, dummies], axis=1) '''
    
fields_to_drop = ['label', 'grade', 'cull']
data = df.drop(fields_to_drop, axis=1)

In [7]:
quant_features = ['rct','tsi', 'stfi', 'caliper','moisture','basisweight']
# Store scalings in a dictionary so we can convert back later
scaled_features = {}
for each in quant_features:
    mean, std = (data[each]).mean(), data[each].std()
    scaled_features[each] = [mean, std]
    data.loc[:, each] = (data[each] - mean)/std
data.head()

,rct,tsi,stfi,caliper,moisture,basisweight
0,-0.459588,-4.702634e-01,-0.125970,-0.418821,-0.389578,-1.331937
1,-0.804279,-2.015414e-01,-0.543143,-0.367641,0.433474,-1.134810
2,-0.287242,5.966826e-15,0.119426,-0.508385,0.227711,-1.124155
3,-0.459588,5.374439e-01,0.233944,-0.406026,-0.266120,-1.092188
4,0.746830,7.389853e-01,-1.999159,0.412850,-1.192053,-0.543430


In [8]:

test_data = data[999:]

data = data[:999]

target_fields = ['rct']
features, targets = data.drop(target_fields, axis=1), data[target_fields]
test_features, test_targets = test_data.drop(target_fields, axis=1), test_data[target_fields]

In [9]:

train_features, train_targets = features[:899], targets[:899]
val_features, val_targets = features[899:], targets[899:]
#features.head()

In [10]:
class NeuralNetwork(object):
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # Set number of nodes in input, hidden and output layers.
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        # Initialize weights
        self.weights_input_to_hidden = np.random.normal(0.0, self.input_nodes**-0.5, 
                                       (self.input_nodes, self.hidden_nodes))

        self.weights_hidden_to_output = np.random.normal(0.0, self.hidden_nodes**-0.5, 
                                       (self.hidden_nodes, self.output_nodes))
        self.lr = learning_rate
        
        self.activation_function = lambda x : 1/(1 + np.exp(-x))   # Replace 0 with your sigmoid calculation.
        
                    
    
    def train(self, features, targets):

        n_records = features.shape[0]
        delta_weights_i_h = np.zeros(self.weights_input_to_hidden.shape)
        delta_weights_h_o = np.zeros(self.weights_hidden_to_output.shape)
        for X, y in zip(features, targets):
            hidden_inputs = np.dot(X,self.weights_input_to_hidden) # signals into hidden layer
            hidden_outputs = self.activation_function(hidden_inputs) # signals from hidden layer
            
            final_inputs = np.dot(hidden_outputs,self.weights_hidden_to_output) # signals into final output layer
            final_outputs = final_inputs # signals from final output layer'this
            
            #### Implement the backward pass here ####
            ### Backward pass ###

            error = y - final_outputs # Output layer error is the difference between desired target and actual output.

            
            output_error_term = error * 1

            hidden_error = np.dot(self.weights_hidden_to_output, error)
            hidden_error_term = hidden_error * hidden_outputs * (1- hidden_outputs)

            delta_weights_i_h += hidden_error_term * X[:,None]
         
            # Weight step (hidden to output)
            hidden_outputs = hidden_outputs[:,None]
            delta_weights_h_o += output_error_term * hidden_outputs
            #print('delta hidden to out: ' + str(delta_weights_h_o))
        self.weights_hidden_to_output += self.lr * delta_weights_h_o/n_records # update hidden-to-output weights with gradient descent step
        self.weights_input_to_hidden += self.lr * delta_weights_i_h/n_records # update input-to-hidden weights with gradient descent step
 
    def run(self, features):

        hidden_inputs = np.dot(features,self.weights_input_to_hidden) # signals into hidden layer
        hidden_outputs = self.activation_function(hidden_inputs) # signals from hidden layer
        
        final_inputs = np.dot(hidden_outputs,self.weights_hidden_to_output) # signals into final output layer
        final_outputs = (final_inputs) # signals from final output layer 
        
        return final_outputs

In [11]:
def MSE(y, Y):
    return np.mean((y-Y)**2)

In [12]:
import sys

### Set the hyperparameters here ###
iterations = 2000
learning_rate = 0.001
hidden_nodes = 3200
output_nodes = 1

N_i = train_features.shape[1]
network = NeuralNetwork(N_i, hidden_nodes, output_nodes, learning_rate)

losses = {'train':[], 'validation':[]}
for ii in range(iterations):
    # Go through a random batch of 128 records from the training data set
    batch = np.random.choice(train_features.index, size=128)
    X, y = train_features.ix[batch].values, train_targets.ix[batch]['rct']
                             
    network.train(X, y)
    
    # Printing out the training progress
    train_loss = MSE(network.run(train_features).T, train_targets['rct'].values)
    val_loss = MSE(network.run(val_features).T, val_targets['rct'].values)
    sys.stdout.write("\rProgress: {:2.1f}".format(100 * ii/float(iterations)) \
                     + "% ... Training loss: " + str(train_loss)[:5] \
                     + " ... Validation loss: " + str(val_loss)[:5])
    sys.stdout.flush()
    
    losses['train'].append(train_loss)
    losses['validation'].append(val_loss)

Progress: 0.2% ... Training loss: 0.778 ... Validation loss: nan

c:\users\lmoyer\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  app.launch_new_instance()
c:\users\lmoyer\appdata\local\programs\python\python36-32\lib\site-packages\numpy\core\fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\lmoyer\appdata\local\programs\python\python36-32\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Progress: 100.0% ... Training loss: 0.055 ... Validation loss: nan

In [13]:
plt.plot(losses['train'], label='Training loss')
plt.plot(losses['validation'], label='Validation loss')
plt.legend()
_ = plt.ylim()

In [14]:
weights_in = []
for each in network.weights_input_to_hidden:
    for weight in each:
        weights_in.append(weight)
    
weights_out = []
for each in network.weights_hidden_to_output:
    for weight in each:
        weights_out.append(weight)

In [15]:
testFeatures = features.reset_index(drop=True)
testTargets = targets.reset_index(drop=True)
mean, std = scaled_features['rct']
columns = ['tsi','stfi','caliper','moisture','basisweight']
newdata = pd.DataFrame(columns=columns)
xs = []
ys = []
tsi = []
stfi = []
caliper = []
moisture = []
basisweight = []
#cull = []
basismean, basisstd = scaled_features['basisweight']
calipermean, caliperstd = scaled_features['caliper']
#cullmean, cullstd = scaled_features['cull']
moisturemean, moisturestd = scaled_features['moisture']
stfimean, stfistd = scaled_features['stfi']
tsimean, tsistd = scaled_features['tsi']
for index, row in features.iterrows():
    prediction = network.run(row).T*std+mean
    xs.append(prediction)
    tsi.append(row['tsi']*tsistd+tsimean)
    stfi.append(row['stfi']*stfistd+stfimean)
    caliper.append(row['caliper']*caliperstd+calipermean)
    moisture.append(row['moisture']*moisturestd+moisturemean)
    basisweight.append(row['basisweight']*basisstd+basismean)
    #cull.append(row['cull']*cullstd+cullmean)

ys = testTargets['rct'].T*std+mean

In [16]:
data.head()

,rct,tsi,stfi,caliper,moisture,basisweight
0,-0.459588,-4.702634e-01,-0.125970,-0.418821,-0.389578,-1.331937
1,-0.804279,-2.015414e-01,-0.543143,-0.367641,0.433474,-1.134810
2,-0.287242,5.966826e-15,0.119426,-0.508385,0.227711,-1.124155
3,-0.459588,5.374439e-01,0.233944,-0.406026,-0.266120,-1.092188
4,0.746830,7.389853e-01,-1.999159,0.412850,-1.192053,-0.543430


In [17]:
from sklearn import linear_model
#train model on data
body_reg = linear_model.LinearRegression()
body_reg.fit(xs, ys)

#visualize results
plt.scatter(xs, ys)
plt.plot(xs, body_reg.predict(xs))
plt.show()
print('r squared value:')
print(body_reg.score(xs,ys))

ModuleNotFoundError: No module named 'sklearn'

In [18]:


bs=[]
for x in xs:
    bs.append(x[0])
newdata['tsi'] = tsi
newdata['stfi'] = stfi
newdata['caliper'] = caliper
newdata['moisture'] = moisture
#newdata['cull'] = cull
newdata['basisweight'] = basisweight
newdata['rct'] = ys
newdata['prediction'] = bs
newdata.to_csv('t1_usp_70_comparison.csv')



In [19]:
import pickle
with open('weight_in_no_grades_usp70', 'wb') as f:
    pickle.dump(network.weights_input_to_hidden, f)
with open('weight_out_no_grades_usp70', 'wb') as f:
    pickle.dump(network.weights_hidden_to_output, f)

import json
# save to file:
with open('variables_usp70.json', 'w') as f:
    json.dump(scaled_features, f)